# 🔍 Pipeline 3: Feature Detection and Description
## Advanced Feature Analysis for Object Recognition

### 🎯 Problem Statement:
**"Detect, describe, and match features in images for object recognition and tracking"**

### 📋 Solution Approach:
1. **Corner Detection** → Find interest points using Harris and FAST
2. **Keypoint Detection** → Extract SIFT and SURF features
3. **Feature Description** → Create robust descriptors
4. **Feature Matching** → Match features between images
5. **Geometric Verification** → Filter matches using RANSAC
6. **Object Recognition** → Identify objects in scene

### 🔧 Techniques Used:
- Harris Corner Detection
- FAST corner detection
- SIFT (Scale-Invariant Feature Transform)
- SURF (Speeded-Up Robust Features)
- ORB (Oriented FAST and Rotated BRIEF)
- Feature matching and homography estimation

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

def show_results(images, titles, rows=2, cols=3, figsize=(15, 10)):
    """Display multiple images in a grid"""
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.flatten() if rows * cols > 1 else [axes]
    
    for i, (img, title) in enumerate(zip(images, titles)):
        if i >= len(axes):
            break
        ax = axes[i]
        
        if len(img.shape) == 3:
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            ax.imshow(img, cmap='gray')
        ax.set_title(title, fontweight='bold')
        ax.axis('off')
    
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

def create_test_images():
    """Create test images with various features"""
    # Image 1: Geometric patterns
    img1 = np.zeros((400, 400, 3), dtype=np.uint8)
    
    # Add rectangles
    cv2.rectangle(img1, (50, 50), (150, 150), (200, 100, 50), -1)
    cv2.rectangle(img1, (200, 80), (350, 180), (100, 200, 150), -1)
    
    # Add circles
    cv2.circle(img1, (100, 250), 50, (150, 50, 200), -1)
    cv2.circle(img1, (300, 300), 30, (50, 150, 100), -1)
    
    # Add triangles
    pts = np.array([[250, 200], [200, 280], [300, 280]], np.int32)
    cv2.fillPoly(img1, [pts], (200, 200, 50))
    
    # Add some texture lines
    for i in range(0, 400, 20):
        cv2.line(img1, (i, 0), (i, 30), (180, 180, 180), 1)
        cv2.line(img1, (0, i), (30, i), (180, 180, 180), 1)
    
    # Image 2: Transformed version
    # Apply rotation and scaling
    center = (200, 200)
    angle = 15
    scale = 0.8
    M = cv2.getRotationMatrix2D(center, angle, scale)
    img2 = cv2.warpAffine(img1, M, (400, 400))
    
    # Add some noise
    noise = np.random.normal(0, 10, img2.shape).astype(np.int16)
    img2 = np.clip(img2.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    
    return img1, img2

def draw_keypoints(img, keypoints, color=(0, 255, 0)):
    """Draw keypoints on image"""
    result = img.copy()
    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        cv2.circle(result, (x, y), 3, color, -1)
        # Draw orientation if available
        if hasattr(kp, 'angle') and kp.angle != -1:
            angle = np.radians(kp.angle)
            x2 = int(x + 10 * np.cos(angle))
            y2 = int(y + 10 * np.sin(angle))
            cv2.line(result, (x, y), (x2, y2), color, 1)
    return result

print("🔍 Feature Detection and Description Pipeline Ready!")

## 📖 Step 1: Load Images and Preprocessing

In [ ]:
# Try to load real images first
try:
    img1 = cv2.imread('D:/FPT_Material/Sem 4/CPV301/Source for PE/Feature Detection and Description/Image/chessboard.jpg')
    img2 = cv2.imread('D:/FPT_Material/Sem 4/CPV301/Source for PE/Feature Detection and Description/Image/home.jpg')
    
    if img1 is None or img2 is None:
        raise FileNotFoundError("Images not found")
    
    # Resize images if they're too large
    max_size = 500
    h1, w1 = img1.shape[:2]
    if max(h1, w1) > max_size:
        scale = max_size / max(h1, w1)
        new_w1, new_h1 = int(w1 * scale), int(h1 * scale)
        img1 = cv2.resize(img1, (new_w1, new_h1))
    
    h2, w2 = img2.shape[:2]
    if max(h2, w2) > max_size:
        scale = max_size / max(h2, w2)
        new_w2, new_h2 = int(w2 * scale), int(h2 * scale)
        img2 = cv2.resize(img2, (new_w2, new_h2))
    
    print("✅ Loaded images from files")
    
except:
    print("⚠️ Creating synthetic test images...")
    img1, img2 = create_test_images()

# Convert to grayscale for feature detection
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

print(f"📊 Image Information:")
print(f"Image 1 shape: {img1.shape}")
print(f"Image 2 shape: {img2.shape}")

# Show original images
show_results(
    [cv2.cvtColor(img1, cv2.COLOR_BGR2RGB), cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)],
    ['Image 1', 'Image 2'],
    rows=1, cols=2, figsize=(12, 6)
)

## 🎯 Step 2: Corner Detection

In [ ]:
# ========================================
# 2.1 Harris Corner Detection
# ========================================

def harris_corner_detection(img, k=0.04, threshold=0.01):
    """Harris corner detection with detailed analysis"""
    # Convert to float
    img_float = np.float32(img)
    
    # Calculate Harris response
    harris_response = cv2.cornerHarris(img_float, 2, 3, k)
    
    # Threshold for corner detection
    corners = harris_response > threshold * harris_response.max()
    
    # Find corner coordinates
    corner_coords = np.where(corners)
    corner_points = list(zip(corner_coords[1], corner_coords[0]))  # (x, y) format
    
    return harris_response, corners, corner_points

# Apply Harris corner detection to both images
harris1, corners1, points1 = harris_corner_detection(gray1)
harris2, corners2, points2 = harris_corner_detection(gray2)

print(f"🎯 Harris Corner Detection Results:")
print(f"Image 1: Found {len(points1)} corners")
print(f"Image 2: Found {len(points2)} corners")

# Visualize Harris corners
img1_harris = img1.copy()
img2_harris = img2.copy()

for x, y in points1:
    cv2.circle(img1_harris, (x, y), 3, (0, 255, 0), -1)

for x, y in points2:
    cv2.circle(img2_harris, (x, y), 3, (0, 255, 0), -1)

# ========================================
# 2.2 FAST Corner Detection
# ========================================

# Create FAST detector
fast = cv2.FastFeatureDetector_create(threshold=50, nonmaxSuppression=True)

# Detect FAST corners
fast_kp1 = fast.detect(gray1, None)
fast_kp2 = fast.detect(gray2, None)

print(f"\n⚡ FAST Corner Detection Results:")
print(f"Image 1: Found {len(fast_kp1)} FAST corners")
print(f"Image 2: Found {len(fast_kp2)} FAST corners")

# Visualize FAST corners
img1_fast = draw_keypoints(img1, fast_kp1, (255, 0, 0))
img2_fast = draw_keypoints(img2, fast_kp2, (255, 0, 0))

# ========================================
# 2.3 goodFeaturesToTrack (Shi-Tomasi)
# ========================================

# Shi-Tomasi corner detection
shi_corners1 = cv2.goodFeaturesToTrack(gray1, maxCorners=100, qualityLevel=0.01, minDistance=10)
shi_corners2 = cv2.goodFeaturesToTrack(gray2, maxCorners=100, qualityLevel=0.01, minDistance=10)

print(f"\n📐 Shi-Tomasi Corner Detection Results:")
if shi_corners1 is not None:
    print(f"Image 1: Found {len(shi_corners1)} Shi-Tomasi corners")
else:
    print(f"Image 1: No Shi-Tomasi corners found")

if shi_corners2 is not None:
    print(f"Image 2: Found {len(shi_corners2)} Shi-Tomasi corners")
else:
    print(f"Image 2: No Shi-Tomasi corners found")

# Visualize Shi-Tomasi corners
img1_shi = img1.copy()
img2_shi = img2.copy()

if shi_corners1 is not None:
    for corner in shi_corners1:
        x, y = corner.ravel().astype(int)
        cv2.circle(img1_shi, (x, y), 3, (0, 0, 255), -1)

if shi_corners2 is not None:
    for corner in shi_corners2:
        x, y = corner.ravel().astype(int)
        cv2.circle(img2_shi, (x, y), 3, (0, 0, 255), -1)

# Show corner detection results
show_results(
    [cv2.cvtColor(img1_harris, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(img1_fast, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(img1_shi, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(img2_harris, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(img2_fast, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(img2_shi, cv2.COLOR_BGR2RGB)],
    ['Harris Corners', 'FAST Corners', 'Shi-Tomasi', 
     'Harris Corners', 'FAST Corners', 'Shi-Tomasi'],
    rows=2, cols=3
)

## 🔍 Step 3: Keypoint Detection and Description

In [ ]:
# ========================================
# 3.1 SIFT Feature Detection
# ========================================

try:
    # Create SIFT detector
    sift = cv2.SIFT_create(nfeatures=500)
    
    # Detect and compute SIFT features
    sift_kp1, sift_desc1 = sift.detectAndCompute(gray1, None)
    sift_kp2, sift_desc2 = sift.detectAndCompute(gray2, None)
    
    print(f"🔍 SIFT Feature Detection Results:")
    print(f"Image 1: {len(sift_kp1)} keypoints, descriptor shape: {sift_desc1.shape if sift_desc1 is not None else 'None'}")
    print(f"Image 2: {len(sift_kp2)} keypoints, descriptor shape: {sift_desc2.shape if sift_desc2 is not None else 'None'}")
    
    sift_available = True
    
except Exception as e:
    print(f"⚠️ SIFT not available: {e}")
    sift_available = False
    sift_kp1, sift_desc1 = [], None
    sift_kp2, sift_desc2 = [], None

# ========================================
# 3.2 ORB Feature Detection
# ========================================

# Create ORB detector
orb = cv2.ORB_create(nfeatures=500)

# Detect and compute ORB features
orb_kp1, orb_desc1 = orb.detectAndCompute(gray1, None)
orb_kp2, orb_desc2 = orb.detectAndCompute(gray2, None)

print(f"\n🎯 ORB Feature Detection Results:")
print(f"Image 1: {len(orb_kp1)} keypoints, descriptor shape: {orb_desc1.shape if orb_desc1 is not None else 'None'}")
print(f"Image 2: {len(orb_kp2)} keypoints, descriptor shape: {orb_desc2.shape if orb_desc2 is not None else 'None'}")

# ========================================
# 3.3 AKAZE Feature Detection
# ========================================

try:
    # Create AKAZE detector
    akaze = cv2.AKAZE_create()
    
    # Detect and compute AKAZE features
    akaze_kp1, akaze_desc1 = akaze.detectAndCompute(gray1, None)
    akaze_kp2, akaze_desc2 = akaze.detectAndCompute(gray2, None)
    
    print(f"\n🌟 AKAZE Feature Detection Results:")
    print(f"Image 1: {len(akaze_kp1)} keypoints, descriptor shape: {akaze_desc1.shape if akaze_desc1 is not None else 'None'}")
    print(f"Image 2: {len(akaze_kp2)} keypoints, descriptor shape: {akaze_desc2.shape if akaze_desc2 is not None else 'None'}")
    
    akaze_available = True
    
except Exception as e:
    print(f"⚠️ AKAZE error: {e}")
    akaze_available = False
    akaze_kp1, akaze_desc1 = [], None
    akaze_kp2, akaze_desc2 = [], None

# ========================================
# 3.4 Visualize Keypoints
# ========================================

# Draw keypoints with orientations and scales
img1_features = []
img2_features = []
feature_names = []

if sift_available and len(sift_kp1) > 0:
    img1_sift = cv2.drawKeypoints(img1, sift_kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    img2_sift = cv2.drawKeypoints(img2, sift_kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    img1_features.extend([img1_sift])
    img2_features.extend([img2_sift])
    feature_names.extend(['SIFT'])

if len(orb_kp1) > 0:
    img1_orb = cv2.drawKeypoints(img1, orb_kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    img2_orb = cv2.drawKeypoints(img2, orb_kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    img1_features.extend([img1_orb])
    img2_features.extend([img2_orb])
    feature_names.extend(['ORB'])

if akaze_available and len(akaze_kp1) > 0:
    img1_akaze = cv2.drawKeypoints(img1, akaze_kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    img2_akaze = cv2.drawKeypoints(img2, akaze_kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    img1_features.extend([img1_akaze])
    img2_features.extend([img2_akaze])
    feature_names.extend(['AKAZE'])

# Show feature detection results
if img1_features:
    all_images = []
    all_titles = []
    
    for i, name in enumerate(feature_names):
        all_images.extend([cv2.cvtColor(img1_features[i], cv2.COLOR_BGR2RGB),
                          cv2.cvtColor(img2_features[i], cv2.COLOR_BGR2RGB)])
        all_titles.extend([f'{name} - Image 1', f'{name} - Image 2'])
    
    show_results(all_images, all_titles, rows=len(feature_names), cols=2, figsize=(12, 4*len(feature_names)))

# ========================================
# 3.5 Feature Analysis
# ========================================

def analyze_keypoints(keypoints, name):
    """Analyze keypoint properties"""
    if len(keypoints) == 0:
        return
    
    # Extract properties
    scales = [kp.size for kp in keypoints]
    responses = [kp.response for kp in keypoints]
    
    print(f"\n📊 {name} Analysis:")
    print(f"  Count: {len(keypoints)}")
    print(f"  Scale range: {min(scales):.1f} - {max(scales):.1f}")
    print(f"  Response range: {min(responses):.3f} - {max(responses):.3f}")
    print(f"  Average scale: {np.mean(scales):.1f}")
    print(f"  Average response: {np.mean(responses):.3f}")

if sift_available:
    analyze_keypoints(sift_kp1, "SIFT Image 1")
    analyze_keypoints(sift_kp2, "SIFT Image 2")

analyze_keypoints(orb_kp1, "ORB Image 1")
analyze_keypoints(orb_kp2, "ORB Image 2")

if akaze_available:
    analyze_keypoints(akaze_kp1, "AKAZE Image 1")
    analyze_keypoints(akaze_kp2, "AKAZE Image 2")

## 🔗 Step 4: Feature Matching

In [ ]:
# ========================================
# 4.1 Brute Force Matcher
# ========================================

def match_features_bf(desc1, desc2, descriptor_type='ORB', ratio_test=True):
    """Match features using Brute Force matcher"""
    if desc1 is None or desc2 is None or len(desc1) == 0 or len(desc2) == 0:
        return []
    
    # Choose appropriate distance metric
    if descriptor_type in ['SIFT', 'SURF', 'AKAZE']:
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    else:  # ORB, BRIEF, etc.
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    
    if ratio_test:
        # Lowe's ratio test
        matches = bf.knnMatch(desc1, desc2, k=2)
        good_matches = []
        for match_pair in matches:
            if len(match_pair) == 2:
                m, n = match_pair
                if m.distance < 0.75 * n.distance:
                    good_matches.append(m)
        return good_matches
    else:
        matches = bf.match(desc1, desc2)
        return sorted(matches, key=lambda x: x.distance)

# ========================================
# 4.2 FLANN Matcher
# ========================================

def match_features_flann(desc1, desc2, descriptor_type='SIFT'):
    """Match features using FLANN matcher"""
    if desc1 is None or desc2 is None or len(desc1) == 0 or len(desc2) == 0:
        return []
    
    if descriptor_type in ['SIFT', 'SURF']:
        # FLANN parameters for SIFT/SURF
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
    else:
        # FLANN parameters for ORB
        FLANN_INDEX_LSH = 6
        index_params = dict(algorithm=FLANN_INDEX_LSH,
                           table_number=6,
                           key_size=12,
                           multi_probe_level=1)
        search_params = dict(checks=50)
    
    try:
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(desc1, desc2, k=2)
        
        # Lowe's ratio test
        good_matches = []
        for match_pair in matches:
            if len(match_pair) == 2:
                m, n = match_pair
                if m.distance < 0.7 * n.distance:
                    good_matches.append(m)
        
        return good_matches
    except Exception as e:
        print(f"FLANN matching failed: {e}")
        return []

# ========================================
# 4.3 Perform Matching for Each Feature Type
# ========================================

matching_results = {}

# SIFT matching
if sift_available and sift_desc1 is not None and sift_desc2 is not None:
    sift_matches_bf = match_features_bf(sift_desc1, sift_desc2, 'SIFT')
    sift_matches_flann = match_features_flann(sift_desc1, sift_desc2, 'SIFT')
    matching_results['SIFT'] = {
        'bf': sift_matches_bf,
        'flann': sift_matches_flann,
        'kp1': sift_kp1,
        'kp2': sift_kp2
    }
    print(f"🔍 SIFT Matching:")
    print(f"  Brute Force: {len(sift_matches_bf)} matches")
    print(f"  FLANN: {len(sift_matches_flann)} matches")

# ORB matching
if orb_desc1 is not None and orb_desc2 is not None:
    orb_matches_bf = match_features_bf(orb_desc1, orb_desc2, 'ORB')
    orb_matches_flann = match_features_flann(orb_desc1, orb_desc2, 'ORB')
    matching_results['ORB'] = {
        'bf': orb_matches_bf,
        'flann': orb_matches_flann,
        'kp1': orb_kp1,
        'kp2': orb_kp2
    }
    print(f"\n🎯 ORB Matching:")
    print(f"  Brute Force: {len(orb_matches_bf)} matches")
    print(f"  FLANN: {len(orb_matches_flann)} matches")

# AKAZE matching
if akaze_available and akaze_desc1 is not None and akaze_desc2 is not None:
    akaze_matches_bf = match_features_bf(akaze_desc1, akaze_desc2, 'AKAZE')
    matching_results['AKAZE'] = {
        'bf': akaze_matches_bf,
        'flann': [],  # FLANN might not work well with AKAZE
        'kp1': akaze_kp1,
        'kp2': akaze_kp2
    }
    print(f"\n🌟 AKAZE Matching:")
    print(f"  Brute Force: {len(akaze_matches_bf)} matches")

# ========================================
# 4.4 Visualize Matches
# ========================================

def draw_matches_custom(img1, kp1, img2, kp2, matches, max_matches=50):
    """Draw matches between two images"""
    # Limit number of matches to display
    matches_to_draw = matches[:max_matches]
    
    # Draw matches
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches_to_draw, None,
                                  flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return img_matches

match_images = []
match_titles = []

for feature_type, results in matching_results.items():
    if len(results['bf']) > 0:
        img_matches_bf = draw_matches_custom(img1, results['kp1'], img2, results['kp2'], results['bf'])
        match_images.append(cv2.cvtColor(img_matches_bf, cv2.COLOR_BGR2RGB))
        match_titles.append(f'{feature_type} - Brute Force')
    
    if len(results['flann']) > 0:
        img_matches_flann = draw_matches_custom(img1, results['kp1'], img2, results['kp2'], results['flann'])
        match_images.append(cv2.cvtColor(img_matches_flann, cv2.COLOR_BGR2RGB))
        match_titles.append(f'{feature_type} - FLANN')

if match_images:
    # Calculate appropriate rows and cols
    n_images = len(match_images)
    cols = min(2, n_images)
    rows = (n_images + cols - 1) // cols
    
    show_results(match_images, match_titles, rows=rows, cols=cols, figsize=(15, 6*rows))
else:
    print("⚠️ No matches found to display")

## 🎯 Step 5: Geometric Verification and Homography

In [ ]:
# ========================================
# 5.1 RANSAC Homography Estimation
# ========================================

def estimate_homography_ransac(kp1, kp2, matches, threshold=5.0):
    """Estimate homography using RANSAC"""
    if len(matches) < 4:
        return None, None, []
    
    # Extract matching points
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # Estimate homography with RANSAC
    homography, mask = cv2.findHomography(src_pts, dst_pts, 
                                         cv2.RANSAC, threshold)
    
    # Get inlier matches
    inlier_matches = [matches[i] for i in range(len(matches)) if mask[i]]
    
    return homography, mask, inlier_matches

# ========================================
# 5.2 Apply RANSAC to Best Matching Results
# ========================================

homography_results = {}

for feature_type, results in matching_results.items():
    matches = results['bf'] if len(results['bf']) > len(results['flann']) else results['flann']
    
    if len(matches) >= 4:
        H, mask, inliers = estimate_homography_ransac(results['kp1'], results['kp2'], matches)
        
        homography_results[feature_type] = {
            'homography': H,
            'inliers': inliers,
            'inlier_ratio': len(inliers) / len(matches) if len(matches) > 0 else 0,
            'kp1': results['kp1'],
            'kp2': results['kp2']
        }
        
        print(f"\n🎯 {feature_type} Homography Results:")
        print(f"  Total matches: {len(matches)}")
        print(f"  Inlier matches: {len(inliers)}")
        print(f"  Inlier ratio: {len(inliers) / len(matches):.2%}")
        
        if H is not None:
            print(f"  Homography matrix:")
            for row in H:
                print(f"    [{row[0]:8.4f} {row[1]:8.4f} {row[2]:8.4f}]")

# ========================================
# 5.3 Visualize Inlier Matches
# ========================================

inlier_images = []
inlier_titles = []

for feature_type, results in homography_results.items():
    if len(results['inliers']) > 0:
        img_inliers = draw_matches_custom(img1, results['kp1'], img2, results['kp2'], 
                                        results['inliers'], max_matches=50)
        inlier_images.append(cv2.cvtColor(img_inliers, cv2.COLOR_BGR2RGB))
        inlier_titles.append(f'{feature_type} - Inlier Matches ({len(results["inliers"])})')

if inlier_images:
    show_results(inlier_images, inlier_titles, 
                rows=len(inlier_images), cols=1, figsize=(15, 6*len(inlier_images)))

# ========================================
# 5.4 Object Detection Using Homography
# ========================================

def detect_object_with_homography(img1, img2, homography):
    """Detect object in img2 using homography from img1"""
    if homography is None:
        return None
    
    # Get corners of img1
    h1, w1 = img1.shape[:2]
    corners1 = np.float32([[0, 0], [w1, 0], [w1, h1], [0, h1]]).reshape(-1, 1, 2)
    
    # Transform corners to img2 coordinates
    corners2 = cv2.perspectiveTransform(corners1, homography)
    
    # Draw the detected object boundary
    img2_with_detection = img2.copy()
    cv2.polylines(img2_with_detection, [np.int32(corners2)], True, (0, 255, 0), 3)
    
    return img2_with_detection, corners2

# Apply object detection for best homography
best_feature_type = None
best_inlier_ratio = 0

for feature_type, results in homography_results.items():
    if results['inlier_ratio'] > best_inlier_ratio and len(results['inliers']) >= 10:
        best_inlier_ratio = results['inlier_ratio']
        best_feature_type = feature_type

if best_feature_type:
    H_best = homography_results[best_feature_type]['homography']
    detection_result = detect_object_with_homography(img1, img2, H_best)
    
    if detection_result:
        img2_detected, corners = detection_result
        
        print(f"\n🎯 Object Detection Results (using {best_feature_type}):")
        print(f"  Best feature type: {best_feature_type}")
        print(f"  Inlier ratio: {best_inlier_ratio:.2%}")
        print(f"  Detected corners in image 2:")
        for i, corner in enumerate(corners):
            print(f"    Corner {i+1}: ({corner[0][0]:.1f}, {corner[0][1]:.1f})")
        
        show_results(
            [cv2.cvtColor(img1, cv2.COLOR_BGR2RGB), cv2.cvtColor(img2_detected, cv2.COLOR_BGR2RGB)],
            ['Template Image', f'Detected Object ({best_feature_type})'],
            rows=1, cols=2, figsize=(15, 6)
        )
else:
    print("\n⚠️ No reliable homography found for object detection")

## 📊 Step 6: Performance Analysis and Summary

In [ ]:
# ========================================
# 6.1 Feature Detection Performance Summary
# ========================================

print("📊 Feature Detection and Matching Performance Summary")
print("=" * 60)

# Create summary table
summary_data = []

for feature_type, results in matching_results.items():
    total_kp1 = len(results['kp1'])
    total_kp2 = len(results['kp2'])
    total_matches = len(results['bf'])
    
    if feature_type in homography_results:
        inliers = len(homography_results[feature_type]['inliers'])
        inlier_ratio = homography_results[feature_type]['inlier_ratio']
        homography_found = homography_results[feature_type]['homography'] is not None
    else:
        inliers = 0
        inlier_ratio = 0
        homography_found = False
    
    summary_data.append({
        'Feature': feature_type,
        'Keypoints 1': total_kp1,
        'Keypoints 2': total_kp2,
        'Total Matches': total_matches,
        'Inliers': inliers,
        'Inlier Ratio': f"{inlier_ratio:.1%}",
        'Homography': '✓' if homography_found else '✗'
    })

# Print summary table
if summary_data:
    # Print header
    headers = ['Feature', 'KP1', 'KP2', 'Matches', 'Inliers', 'Ratio', 'Homography']
    print(f"{'Feature':<8} {'KP1':<5} {'KP2':<5} {'Matches':<8} {'Inliers':<8} {'Ratio':<8} {'Homography':<10}")
    print("-" * 60)
    
    for data in summary_data:
        print(f"{data['Feature']:<8} {data['Keypoints 1']:<5} {data['Keypoints 2']:<5} "
              f"{data['Total Matches']:<8} {data['Inliers']:<8} {data['Inlier Ratio']:<8} {data['Homography']:<10}")

# ========================================
# 6.2 Feature Quality Analysis
# ========================================

print(f"\n🔍 Feature Quality Analysis:")
print("-" * 40)

# Analyze repeatability and matching performance
for feature_type, results in matching_results.items():
    kp1_count = len(results['kp1'])
    kp2_count = len(results['kp2'])
    matches_count = len(results['bf'])
    
    if kp1_count > 0 and kp2_count > 0:
        matching_rate = matches_count / min(kp1_count, kp2_count)
        print(f"\n{feature_type}:")
        print(f"  Keypoint density: {(kp1_count + kp2_count) / 2:.1f} per image")
        print(f"  Matching rate: {matching_rate:.1%}")
        
        if feature_type in homography_results:
            inlier_ratio = homography_results[feature_type]['inlier_ratio']
            print(f"  Geometric consistency: {inlier_ratio:.1%}")
            
            # Overall score (combination of matching rate and geometric consistency)
            overall_score = (matching_rate * 0.3 + inlier_ratio * 0.7) * 100
            print(f"  Overall quality score: {overall_score:.1f}/100")

# ========================================
# 6.3 Recommendations
# ========================================

print(f"\n💡 Recommendations for Feature Detection:")
print("-" * 50)

if best_feature_type:
    print(f"✅ Best performing method: {best_feature_type}")
    print(f"   Reason: Highest inlier ratio ({best_inlier_ratio:.1%}) with sufficient matches")

print(f"\n📚 Method Characteristics:")
if sift_available:
    print(f"• SIFT: Scale and rotation invariant, best for textured images")
print(f"• ORB: Fast, good for real-time applications, binary descriptors")
if akaze_available:
    print(f"• AKAZE: Good balance of speed and accuracy, nonlinear scale space")

print(f"\n🎯 Use Case Guidelines:")
print(f"• Real-time applications: Use ORB")
print(f"• High accuracy needed: Use SIFT (if available)")
print(f"• Planar objects: Any method works well")
print(f"• Textured scenes: SIFT or AKAZE preferred")
print(f"• Low-texture scenes: Increase detector thresholds")

print(f"\n🔧 Parameter Tuning Tips:")
print(f"• Increase nfeatures for more keypoints")
print(f"• Lower thresholds for more features (but more noise)")
print(f"• Use ratio test (0.7-0.8) for better matches")
print(f"• RANSAC threshold: 1-5 pixels typical")

print(f"\n🎉 Feature Detection Pipeline Completed Successfully!")
print(f"📈 This pipeline demonstrated:")
print(f"   - Multiple feature detection algorithms")
print(f"   - Different matching strategies")
print(f"   - Geometric verification with RANSAC")
print(f"   - Object detection using homography")
print(f"   - Performance analysis and comparison")